In [1]:
from calendar import month
from operator import index
import snscrape.modules.twitter as sntwitter
import pandas as pd
import glob
import datetime

release_dates = pd.read_csv("Release dates - data.csv")
release_dates_map = {}

folderpathrt = "rotten_info/*"
filesrt = []

for file in glob.glob(folderpathrt):
    filesrt.append(file)

for file in filesrt:
    print(file)
    movies = pd.read_csv(file)

    tweets = []
    for ind, movie in movies.iterrows():
        movien = movie['Release Group']
        if not pd.isna(movie['Release Date (Theaters)']):
            release_date = ('').join(movie['Release Date (Theaters)'].strip('[]').split(',')).replace("'", "")
            rd = datetime.datetime.strptime(release_date, "%b %d %Y")
            # if release_dates_map.get(movien.lower(), ''):
            release_dates_map[movien.lower()] = rd

folderpath = "Box Office Mojo/*"
files = []

for file in glob.glob(folderpath):
    files.append(file)

for ind, row in release_dates.iterrows():
    if row['Release-Date']:
        rd = datetime.datetime.strptime(row['Release-Date'], "%Y-%m-%d")
        if not release_dates_map.get(row['Movie'].lower(), None):
            release_dates_map[row['Movie'].lower()] = rd
    
count = 0
c = 0
for file in files:
    if '2017' in file:
        print(file)
        movies = pd.read_csv(file)

        tweets = []
        for ind, movie in movies.iterrows():
            c += 1
            query_txt = movie['Release Group']
            release_date = release_dates_map.get(movie['Release Group'].lower(), None)
            if release_date:
                count += 1
                # rd = datetime.datetime.strptime(release_date, "%Y-%m-%d")
                since = release_date - datetime.timedelta(weeks=4)
                until = release_date + datetime.timedelta(weeks=1)
                query_txt += " since:%s until:%s" % (since.date(), until.date())
                print(movie['Release Group'], release_date.date())

                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query_txt).get_items()):
                    if i>5000:
                        break
                    tweets.append([movie['Release Group'], tweet.date, tweet.id, tweet.content, tweet.user.username])

        tweets_df = pd.DataFrame(tweets, columns=['Movie Name', 'Datetime', 'Tweet Id', 'Text', 'Username'])
        tweets_df.to_csv("Twitter/tweets_%s" % file.split('/')[1], index=False, escapechar=r'|')
        break


Box Office Mojo/2017.csv
Star Wars: Episode VIII - The Last Jedi 2017-12-15
Beauty and the Beast 2017-03-17
The Fate of the Furious 2017-04-14
Despicable Me 3 2017-06-30
Jumanji: Welcome to the Jungle 2017-12-20
Spider-Man: Homecoming 2017-07-07
Wolf Warrior 2 2017-07-28
Guardians of the Galaxy Vol. 2 2017-05-05
Thor: Ragnarok 2017-11-03
Wonder Woman 2017-06-02
Coco 2017-11-22
Pirates of the Caribbean: Dead Men Tell No Tales 2017-05-26
It 2017-09-08
Justice League 2017-11-17
Logan 2017-03-03
Transformers: The Last Knight 2017-06-21
Kong: Skull Island 2017-03-10
The Boss Baby 2017-03-31
Dunkirk 2017-07-21
War for the Planet of the Apes 2017-07-14
The Greatest Showman 2017-12-20
Kingsman: The Golden Circle 2017-09-22
The Mummy 2017-06-09
Cars 3 2017-06-16
Fifty Shades Darker 2017-02-10
Murder on the Orient Express 2017-11-10
xXx: Return of Xander Cage 2017-01-20
The Lego Batman Movie 2017-02-10
Annabelle: Creation 2017-08-11
Wonder 2017-11-17
Ferdinand 2017-12-15
Split 2017-01-20
Blade R